In [26]:
!pip install --upgrade --quiet  langchain-community
!pip install --upgrade --quiet langchainhub langgraph
!pip install -qU langchain-openai

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import sqlite3
import os
import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

def build_connection_string():
    """Build the connection string"""
    try:
        protocol = os.getenv("DB_PROTOCOL")
        user = os.getenv("DB_USER")
        password = os.getenv("DB_PASSWORD")
        host = os.getenv("DB_HOST")
        port = os.getenv("DB_PORT")
        database = os.getenv("DB_DATABASE")
        schema = os.getenv("DB_SCHEMA")
        driver = os.getenv("DB_DRIVER")

        conn_string = f"{protocol}://{user}:{password}@{host}:{port}/{database}?options=-c%20search_path={schema}"
        if driver:
            conn_string += f"?driver={driver}"
        return conn_string
    except Exception as exc:
        raise exc

def create_postgres_engine():
    """Creates the db engine according to env runtime"""
    conn_string = build_connection_string()
    engine = create_engine(conn_string, connect_args={"keepalives_idle": 30})
    return engine

def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )

engine_postgres = create_postgres_engine()
engine = get_engine_for_chinook_db()

db = SQLDatabase(engine_postgres)

In [3]:
from langchain_openai import AzureChatOpenAI
import os

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [4]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [5]:
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a0efabb9e20>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a0efabb9e20>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7a0efabb9e20>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [6]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [7]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

In [9]:
# example_query = """Por padrão todas as tabelas do banco de dados iniciam com o nome do schema. 
# As tabelas do galaxia possuem em seu nome o termo 'glx'. 
# 
# Desta forma, quantas series nós temos cadastrados no banco de dados, utilize apenas o schema 'tru' na sua análise?
# """

example_query = """Por padrão todas as tabelas do banco de dados iniciam com o nome do schema. 
As tabelas do galaxia possuem em seu nome o termo 'glx'. 

Desta forma, qual as 5 séries, que ainda estão em andamento, com o vencimento mais distante, utilize apenas o schema 'tru' na sua análise. 
Descarte de sua análise registros que não estão preenchidos.
Retorne apenas o código if e a data de vencimento delas, nada a mais.  
"""

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Por padrão todas as tabelas do banco de dados iniciam com o nome do schema. 
As tabelas do galaxia possuem em seu nome o termo 'glx'. 

Desta forma, qual as 5 séries, que ainda estão em andamento, com o vencimento mais distante, utilize apenas o schema 'tru' na sua análise. 
Descarte de sua análise registros que não estão preenchidos.
Retorne apenas o código if e a data de vencimento delas, nada a mais.  

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_DsVznUqd0Dn3QgSHJYK3RN1W)
 Call ID: call_DsVznUqd0Dn3QgSHJYK3RN1W
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

rfd_ativos_status_cvm, rfd_tb_agentes_fiduciarios, rfd_tb_assembleia, rfd_tb_ativos_thor, rfd_tb_auditoria_carteira_xp, rfd_tb_cnab_categoria_lancamento, rfd_tb_cnab_forma_lancamento, rfd_tb_cnab_instrucao_m